In [ ]:
import requests
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

In [ ]:
stats=pd.read_excel('statid.xlsx')

In [ ]:
games=[]
for j in range(1,18):
    print(j)
    url='http://www.nfl.com/ajax/scorestrip?season={}&seasonType={}&week={}'.format(2018,"REG",j)



    root=ET.fromstring(requests.get(url).text)


    for i in root[0]:
        games.append(i.attrib['eid'])

In [ ]:
url='http://www.nfl.com/ajax/scorestrip?season={}&seasonType={}&week={}'.format(2018,"REG",1)
url

In [ ]:
games=['2010121212']

In [ ]:
gid=2018091000
# for gid in games:
url2='http://www.nfl.com/liveupdate/game-center/{}/{}_gtd.json'.format(gid,gid)

d=requests.get(url2).json()

df_d=[]
for key,val in d[str(gid)]['drives'].items():
    if key =='crntdrv':
        continue
    df=pd.DataFrame(val)[['start','end']].dropna()

    df2=pd.DataFrame([0])
    counter=0
    for i in df.columns:
        counter=0
        for j in df[i]:
            df2[i+' '+df[i].index[counter]]=j
            counter+=1
    df2=df2.drop([0,'start team','end team','end qtr'],axis=1)



    df=pd.DataFrame(val).drop(['plays','start','end'],axis=1).drop_duplicates()
    try:
        df=df.drop('redzone',axis=1)
    except:
        pass

    df=df.reset_index().drop('index',axis=1).reset_index()

    df2=df2.reset_index()

    df=pd.merge(df,df2,on='index').drop('index',axis=1)
    df['driveid']=key
    df_d.append(df)



In [ ]:
df

In [ ]:
df.pivot_table(index='defteam',aggfunc=sum,values='points')

In [ ]:
piv=df3.pivot_table(index='playerName',aggfunc=sum,values='points').reset_index()
piv[(piv['playerName']=='NYJ')|(piv['playerName']=='DET')]

In [ ]:
piv

In [ ]:
# gid=2018120900
dfs=[]
# for gid in games:
#     print (gid)
#     url2='http://www.nfl.com/liveupdate/game-center/{}/{}_gtd.json'.format(gid,gid)

#     d=requests.get(url2).json()


#     for key,val in d[str(gid)]['drives'].items():
#         if key =='crntdrv':
#             continue

#         for key2,val2 in val['plays'].items():
#             for key3,val3 in val2['players'].items():
#                 df=(pd.DataFrame(val3))
#                 df['playerid']=key3
#                 df['play']=key2
#                 df['drive']=key
#                 df['description']=val2['desc']
#                 df['gid']=gid
#                 df['posteam']=val2['posteam']
#                 dfs.append(df)
# df=pd.concat(dfs)



# print (gid)
# url2='http://www.nfl.com/liveupdate/game-center/{}/{}_gtd.json'.format(gid,gid)

# d=requests.get(url2).json()


for key,val in d[str(gid)]['drives'].items():
    if key =='crntdrv':
        continue

    for key2,val2 in val['plays'].items():
        for key3,val3 in val2['players'].items():
            df=(pd.DataFrame(val3))
            df['playerid']=key3
            df['play']=key2
            df['drive']=key
            df['description']=val2['desc']
            df['posteam']=val2['posteam']
            dfs.append(df)
df=pd.concat(dfs)

In [ ]:
url2

In [ ]:
# df=pd.merge(df,stats,left_on='statId',right_on='id',how='left')

# df['play']=df['play'].astype(np.int64)

# df['points']=df['yds points'].fillna(0)*df['yards'].fillna(0)+df['existence points'].fillna(0)

# df['playerName'] = np.where(df['cat']=='defense', df['clubcode'],df['playerName'])

# piv=df.pivot_table(index='playerName',values=['points','yards'],aggfunc=sum)

# df.to_clipboard(sep='\t')

In [ ]:
import statid

statdfs=[]
for key,val in statid.idmap.items():
    tdf=pd.DataFrame(val)
    tdf['statId']=key
    statdfs.append(tdf)

st=pd.concat(statdfs)

df2=pd.merge(df,st,on='statId',how='left')

df3=pd.merge(df2,pd.read_excel('points.xlsx'),
         left_on=['cat','fields'],
         right_on=['cat2','field']).\
drop({'cat_x','fields','cat2'},axis=1).\
    rename({'cat_y':'cat'},axis=1)

df3['points']=np.where(df3['cat']=='yds',df3['points']*df3['yards'],df3['points'])

df3['playerName']=np.where(df3['clubcode']==df3['posteam'],df3['playerName'],df3['clubcode'])

In [ ]:
df3.to_csv('2018 Season.csv',index=False)

In [1]:
import requests
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

In [2]:
games=[]
for j in range(1,2):
    print(j)
    url='http://www.nfl.com/ajax/scorestrip?season={}&seasonType={}&week={}'.format(2018,"REG",j)



    root=ET.fromstring(requests.get(url).text)


    for i in root[0]:
        games.append(i.attrib['eid'])

1


In [2]:
games=['2010121212']

In [3]:
import statid
# gid=2018120900
df_g=[]
drives=[]
for gid in games:
    print (gid)
    url2='http://www.nfl.com/liveupdate/game-center/{}/{}_gtd.json'.format(gid,gid)

    d=requests.get(url2).json()

    df_d=[]
    for key,val in d[str(gid)]['drives'].items():
        
        if key =='crntdrv':
            continue
        df=pd.DataFrame(val)[['start','end']].dropna()

        df2=pd.DataFrame([0])
        counter=0
        for i in df.columns:
            counter=0
            for j in df[i]:
                df2[i+' '+df[i].index[counter]]=j
                counter+=1
        df2=df2.drop([0,'start team','end team','end qtr'],axis=1)



        df=pd.DataFrame(val).drop(['plays','start','end'],axis=1).drop_duplicates()
        try:
            df=df.drop('redzone',axis=1)
        except:
            pass

        df=df.reset_index().drop('index',axis=1).reset_index()

        df2=df2.reset_index()

        df=pd.merge(df,df2,on='index').drop('index',axis=1)
        df['driveid']=key
        df_d.append(df)

        for key2,val2 in val['plays'].items():
            for key3,val3 in val2['players'].items():
                df=(pd.DataFrame(val3))
                df['playerid']=key3
                df['play']=key2
                df['drive']=key
                df['description']=val2['desc']
                df['gid']=gid
                df['posteam']=val2['posteam']
                df_g.append(df)
                
                
    df=pd.concat(df_d).fillna(0)

    df['posteam']=df['posteam'].str.pad(3,'right')
    df['start yrdln']=np.where(df['start yrdln']=='',df['posteam']+' 0',df['start yrdln'])
    df['start yrdln']=np.where(df['start yrdln']=='50',df['posteam']+' 50',df['start yrdln'])
    df['start yrdln']=df['start yrdln'].str.replace('  ',' ')

    df['start yrdln']=np.where(df['posteam']==df['start yrdln'].str[:3].str.strip(),
                               df['start yrdln'].str.split(' ').str[1],
                               (50-df['start yrdln'].str.split(' ').str[1].astype(np.int64))+50)

    df['end yrdln']=df['start yrdln'].astype(np.int64)+df['penyds'].astype(np.int64)+df['ydsgained'].astype(np.int64)
    df['gid']=gid



    dteam={d[str(gid)]['home']['abbr']:d[str(gid)]['away']['abbr'],
           d[str(gid)]['away']['abbr']:d[str(gid)]['home']['abbr'],
          '':''}

    df['defteam']=df['posteam'].str.strip().apply(lambda x: dteam[x])

    df['points']=0

    df['points']=np.where(df['fds']>0,df['points']-(.2*df['fds']),df['points'])

    df['points']=np.where(df['result']=='Touchdown',
                          np.where(df['start yrdln'].astype(np.int64)<50,
                                   df['points']-3,
                                   df['points']-1),df['points'])

    df['points']=np.where(df['result']=='Field Goal',
                          np.where(df['start yrdln'].astype(np.int64)<50,
                                   df['points']-1,
                                   df['points']-0),df['points'])

    df['points']=np.where((df['fds']==0)&((df['result']=='Punt')|
                                         (df['result']=='Fumble')|
                                         (df['result']=='Interception')),
                          df['points']+1,
                          df['points'])
    df['points']=np.where((df['result']=='Downs'),
                          df['points']+1,
                          df['points'])


    drives.append(df.copy())
df=pd.concat(df_g)
statdfs=[]
for key,val in statid.idmap.items():
    tdf=pd.DataFrame(val)
    tdf['statId']=key
    statdfs.append(tdf)

st=pd.concat(statdfs)

df2=pd.merge(df,st,on='statId',how='left')

# df3=pd.merge(df2,pd.read_excel('points.xlsx'),
#          left_on=['cat','fields'],
#          right_on=['cat2','field']).\
# drop({'cat_x','fields','cat2'},axis=1).\
#     rename({'cat_y':'cat'},axis=1)

df3=pd.merge(df2,pd.read_excel('points.xlsx'),
         left_on=['fields'],
         right_on=['field']).\
drop({'cat_x','fields','cat2'},axis=1).\
    rename({'cat_y':'cat'},axis=1)

df3['points']=np.where(df3['cat']=='yds',df3['points']*df3['yards'],df3['points'])
df3['playerName']=df3['playerName'].fillna(1)
df3['playerName']=np.where(df3['clubcode']==df3['posteam'],df3['playerName'],df3['clubcode'])
df3['playerName']=np.where(df3['playerName']==1,
                           np.where(df3['cat']=='defense',
                                    df3['posteam'].str.strip().apply(lambda x: dteam[x]),
                                    df3['clubcode']),
                          df3['playerName'])
plays=df3.copy()
drives=pd.concat(drives)

2010121212


In [4]:
plays.to_clipboard(sep='\t')

In [5]:
drives.to_clipboard(sep='\t')

In [73]:
df3['playerName'].fillna(1)

0                IND
1                IND
2                BUF
3         S.Hauschka
4       C.McLaughlin
5          C.Hedlund
6         S.Hauschka
7         S.Hauschka
8         S.Hauschka
9          R.Sanchez
10      C.McLaughlin
11         C.Hedlund
12         C.Hedlund
13      C.McLaughlin
14         C.Hedlund
15        S.Hauschka
16      C.McLaughlin
17         C.Hedlund
18        S.Hauschka
19        S.Hauschka
20        S.Hauschka
21         R.Sanchez
22      C.McLaughlin
23         C.Hedlund
24         C.Hedlund
25      C.McLaughlin
26         C.Hedlund
27                 1
28                 1
29                 1
            ...     
1093               1
1094        T.Yeldon
1095             IND
1096        P.Walker
1097             BUF
1098             BUF
1099             IND
1100             IND
1101             BUF
1102               1
1103               1
1104             BUF
1105             IND
1106             IND
1107         S.Perry
1108         C.Kelly
1109         

In [67]:
z=df3.iloc[1010,1]

In [8]:
# plays['dupeindex']=plays['playerName']+' ' + plays['play'].astype(str)+' ' + plays['statId'].astype(str)+' '+plays['drive']+' '+ plays['gid']+' '+ plays['field']

In [9]:
# plays[plays['dupeindex'].duplicated(keep=False)].to_clipboard(sep='\t')

In [23]:
df2[df2['statId']==6]

,clubcode,playerName,sequence,statId,yards,playerid,play,drive,description,gid,posteam,cat,fields,desc,long
65,BUF,None,2,6,0.0,0,338,2,(11:59) (Shotgun) J.Allen scrambles left tackl...,2019080853,BUF,team,third_down_att,3rd down attempt converted,3rd down play resulted in a first down or touc...
66,BUF,None,2,6,0.0,0,338,2,(11:59) (Shotgun) J.Allen scrambles left tackl...,2019080853,BUF,team,third_down_conv,3rd down attempt converted,3rd down play resulted in a first down or touc...
67,BUF,None,2,6,0.0,0,338,2,(11:59) (Shotgun) J.Allen scrambles left tackl...,2019080853,BUF,defense,third_down_stop_failed,3rd down attempt converted,3rd down play resulted in a first down or touc...
136,BUF,None,2,6,0.0,0,606,4,(7:13) (Shotgun) J.Allen pass short left to Z....,2019080853,BUF,team,third_down_att,3rd down attempt converted,3rd down play resulted in a first down or touc...
137,BUF,None,2,6,0.0,0,606,4,(7:13) (Shotgun) J.Allen pass short left to Z....,2019080853,BUF,team,third_down_conv,3rd down attempt converted,3rd down play resulted in a first down or touc...
138,BUF,None,2,6,0.0,0,606,4,(7:13) (Shotgun) J.Allen pass short left to Z....,2019080853,BUF,defense,third_down_stop_failed,3rd down attempt converted,3rd down play resulted in a first down or touc...
304,BUF,None,2,6,0.0,0,1345,8,(12:20) D.Singletary left guard to IND 19 for ...,2019080853,BUF,team,third_down_att,3rd down attempt converted,3rd down play resulted in a first down or touc...
305,BUF,None,2,6,0.0,0,1345,8,(12:20) D.Singletary left guard to IND 19 for ...,2019080853,BUF,team,third_down_conv,3rd down attempt converted,3rd down play resulted in a first down or touc...
306,BUF,None,2,6,0.0,0,1345,8,(12:20) D.Singletary left guard to IND 19 for ...,2019080853,BUF,defense,third_down_stop_failed,3rd down attempt converted,3rd down play resulted in a first down or touc...
321,BUF,None,2,6,0.0,0,1409,8,(11:00) (Shotgun) M.Barkley pass short left to...,2019080853,BUF,team,third_down_att,3rd down attempt converted,3rd down play resulted in a first down or touc...


In [11]:
play_piv=plays.pivot_table(index=['clubcode', 'playerName', 'statId', 'yards',
       'play', 'drive', 'description', 'gid', 'posteam', 'cat',
       'field'],aggfunc={'playerid':lambda x: ','.join(x),
                                               'points':np.mean}).reset_index()

pp2=play_piv.pivot_table(index=['playerName','clubcode','gid'],values='points',aggfunc=sum).reset_index()

dpiv=drives.pivot_table(index=['defteam','defteam','gid'],values='points',aggfunc=sum).reset_index()

dpiv.columns=pp2.columns

fullpiv=pd.concat([pp2,dpiv]).pivot_table(index=['playerName','clubcode','gid'],values='points',aggfunc=sum).reset_index()

In [ ]:
fullpiv.to_clipboard(sep='\t')

In [ ]:
drives[drives['defteam']=='MIN'].to_clipboard(sep='\t')

In [ ]:
plays[plays['playerName']=='MIN'].dropna().pivot_table(index='field',values='points',aggfunc=sum).to_clipboard()

In [ ]:
plays[plays['playerName']=='MIN'].dropna()